# Workflow percepcion remota utilizando imagenes Landsat y Earth engine

1. Descargar Anaconda

2. Crear virtual environment
    > Posibles problemas: **conda activate no es reconocido por VScode**. Ejecutar anaconda prompt y escribir `conda init powershell`
                         > **La ejecución de scripts no está habilidatada**. Abrir windows powershell y ejecutar Set-ExecutionPolicy, opción 1.
3. installar librerias earthengine-api y geemap:
    > `conda install -c conda-forge earthengine-api`
    > `conda install -c conda-forge geemap`

4. Instalar Google CLI (https://cloud.google.com/sdk/docs/install):
    > `(New-Object Net.WebClient).DownloadFile("https://dl.google.com/dl/cloudsdk/channels/rapid/GoogleCloudSDKInstaller.exe", "$env:Temp\GoogleCloudSDKInstaller.exe")`
    > `& $env:Temp\GoogleCloudSDKInstaller.exe`

5. Configurar gcloud CLI (último paso de la instalación):
    > Conectar con cuenta y proyecto (**rs_diss**)

6. Autentificar earthengine `earthengine authenticate`
    > Generar token e introducirlo (Se debe habilitar la API de Earth engine en el proyecto de Google Cloud SDK)

7. Definir area de estudio

8. Descargar imagenes

9. Descartar imagenes con base en su nivel de nubosidad

10. Acotar imagenes a una region mas pequeña (zoom)

11. Aplicar analisis


In [ ]:
# Librerias

# Manipulacion 
import pandas as pd 
import geopandas as gpd

# Earth engine (descarga de imagenes)
import ee 

# Mapas interactivos
import folium

# Plot
from IPython.display import Image

In [ ]:
# Autentificar Earth engine
# ee.Authenticate() Esto solo debe ser ejecutado la primera vez

ee.Initialize()

In [ ]:
# Definir area de estudio

# Coordenadas
lat = 21.8833333
lon = -102.3

# Punto de interes
poi = ee.Geometry.Point(lon, lat)

# Periodo de tiempo
start_date = "2021-11-01"
end_date = "2022-11-15"

In [ ]:
# Descarga de imagenes Landsat 9
landsat = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")\
    .filterBounds(poi)\
    .filterDate(start_date, end_date)

# Cantidad de imagenes obtenidas
print("Imagenes totales:", landsat.size().getInfo())

In [ ]:
# Informacion de la primera imagen
landsat.first().getInfo()

In [ ]:
# Nubosidad
print(landsat.first().get("CLOUD_COVER").getInfo())

In [ ]:
# Fechas
landsat.first().get("DATE_ACQUIRED").getInfo()

In [ ]:
# Bandas 
landsat.first().bandNames().getInfo()

In [ ]:
# Parametros para preprocesamiento
parameters = {
    "min":7000,
    "max":16000,
    "dimensions":800, # Tamaño cuadrado en pixeles
    "bands": ["SR_B4", "SR_B3", "SR_B2"] # (R, G, B)
}

In [ ]:
# Imagenes en lista
landsat_list = landsat.toList(landsat.size())

# Funcion para mostrar cada imagen
def display_img(ids, parameters, ndvi_display = False):

    for i in ids:
    
        # Fecha de captura
        date = ee.Image(landsat_list.get(i)).get("DATE_ACQUIRED").getInfo()

        # Nubosidad
        cloud = ee.Image(landsat_list.get(i)).get("CLOUD_COVER").getInfo()
    
        # Informacion de la imagen
        print("Imagen #", i, date, "Nubosidad:", cloud)

        if ndvi_display == True:
            display(Image(url = ee.Image(landsat_list.get(i)).normalizedDifference(["SR_B5", "SR_B4"]).getThumbURL(parameters)))

        else:
            # Plot
            display(Image(url = ee.Image(landsat_list.get(i)).getThumbURL(parameters)))
        
# Funcion para crear data frame con info de las imagenes
def img_info(ids):
    data = []
    for i in ids:

        # Fecha de captura
        date = ee.Image(landsat_list.get(i)).get("DATE_ACQUIRED").getInfo()

        # Nubosidad
        cloud = ee.Image(landsat_list.get(i)).get("CLOUD_COVER").getInfo()

        # Informacion de la imagen
        print("Imagen #", i, date, "Nubosidad:", cloud)
        
        image_data = [i, date, cloud]
        data.append(image_data)
    
    return data

In [ ]:
# DF de informacion
landsat_df = pd.DataFrame(img_info(range(landsat.size().getInfo())), columns = ["img_id", "date", "cloud_cover"])

In [ ]:
# Imagenes recolectadas
display_img(landsat_df.img_id, parameters = parameters)

In [ ]:
# Data frame con  informacion de las imagenes
landsat_df.head() # Con esto se puede filtrar por la nubosidad, la cual parece ser buena por debajo de 10

## Filtrado de imagenes

In [ ]:
# Seleccionar las imagenes que tengan una nubosidad por debajo de 10
ids_noClouds = landsat_df.img_id[landsat_df.cloud_cover < 10]
len(ids_noClouds) # 23 Imagenes 

In [ ]:
# Definir region de interes
roi = poi.buffer(10000) # Metros = 10 km

In [ ]:
# Nuevos parametros
parameters = {
    "min":7000,
    "max":16000,
    "dimensions":800, # Tamaño cuadrado en pixeles
    "bands": ["SR_B4", "SR_B3", "SR_B2"], # (R, G, B)
    "region":roi # Region de interes
}

In [ ]:
display_img([0, 2], parameters)

In [ ]:
# Imagenes con zoom en la ciudad de Ags
display_img(ids_noClouds, parameters = parameters)

## NDVI

$NDVI = \frac{NIR - RED}{NIR + RED}$

* Saludable = Valores altos
* No saludable = Valores bajos

`ndvi = image.normalizedDifference(["B5", "B4"])`

In [ ]:
# Parametros para el analisis NDVI
palette = ["red", "yellow", "green"]

ndvi_parameters = {
    "min":0, # Este color sera rojo - no saludable
    "max":0.4, # Lo que sea mayor sera verde - saludable
    "dimensions":512,
    "palette":palette,
    "region":roi
}

In [ ]:
# NDVI index por imagen
display_img(ids_noClouds, ndvi_parameters, ndvi_display = True)

* GOES - Satélite
* Repetir para Sentinel
* aplicar índice de agua (revisar variaciones)
* NOT - Lab Observatorio tierra (Revisar)
* Sensores de humedad en satélites (revisar nombre)
* Utilizar shape de cuerpos de agua. Contrastar el shapefile con las imágenes. Serie de INEGI de cobertura de suelo. 

1. Bitácora de fugas (fecha y localización)
2. Imágenes de drones (o información al respecto)